#Exploring AJILE12 dataset

It can get cumbersome to manually dissect an NWB file with print statements. There are a few ways to view an NWB graphically instead. A great way to do this in a Jupyter notebook is with **[NWBWidgets](https://github.com/NeurodataWithoutBorders/nwbwidgets)**. Here, you can use NWBWidgets to view a file from a location on your machine. If you don't want to download a file just to view it, you can still use NWBWidgets to view it remotely. Check out [Streaming an NWB File with fsspec](./stream_nwb.ipynb) to learn how to do this. Another way to explore an NWB file, that doesn't require Jupyter, is with [HDFView](https://www.hdfgroup.org/downloads/hdfview/)

## Environment Setup

This code is meant to be run in a Google Colab notebook. It checks if the notebook is running on a Google Colab GPU, and if it is, it clones the Neuromatch-AJILE12 repository from GitHub, changes the current working directory to the Neuromatch-AJILE12k directory, and installs the package in editable mode using pip. This is done to set up the environment for using the Neuromatch-AJILE12 package in a Google Colab notebook.

In [ ]:
### if running on Google Colab, run this cell once, then restart the runtime and run the rest of the notebook
import os
if "COLAB_GPU" in os.environ:
    !git clone https://github.com/neurovium/Neuromatch-AJILE12
    %cd Neuromatch-AJILE12
    %pip install -e .